# BIDMC Datathon Question #1
# English vs. Non-English Speaker MIMIC-III Cohort

# Notebook 1: Explore database and identify English vs. Non-English speakers

The aim of this notebook is to access the [Medical Information Mart for Intensive Care III (MIMIC-III)](https://mimic.physionet.org/) database and to start generating the cohort to be investigated as part of BIDMC Datathon Question #1 assessing whether non-English speakers receive different levels of intensive care at the end of life than English speakers.

This notebook is the R version.

# Setup

## Prerequisites

- If you do not have a Gmail account, please create one at http://www.gmail.com. 
- If you have not yet signed the data use agreement (DUA) sent by the organizers, please do so now to get access to the dataset.

## Load libraries

Run the following cells to install and import some libraries.

In [154]:
install.packages("bigrquery")
install.packages("googledrive")
install.packages("R.utils")
install.packages("cowplot")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [0]:
suppressPackageStartupMessages({
  library(tidyverse)   ## for easy DS in R
  library(cowplot)     ## for plotting plots together
  library(bigrquery)   ## for querying BigQuery
  library(googledrive) ## for read/write Google Drive
  library(httr)        ## for hacking bigrquery to run in Colab
  library(R.utils)     ## ""
  library(repr)        ## for modifying R plot dims in Colab
})

We also set a few global parameters for nicer plots.

In [0]:
## specify default plot dimensions
options(repr.plot.width=4, repr.plot.height=3, repr.plot.res = 300)

## specify default ggplot2 theme
theme_set(theme_bw())

## Connect to BigQuery

Before running any queries, you need to first authenticate yourself by running the following cell. If you are running it for the first time, it will ask you to follow a link to log in using your Gmail account, and accept the data access requests to your profile. Once this is done, it will generate a string of verification code, which you should paste back to the cell below and press enter.

The following is a hack to allow R access to Google BigQuery with the `bigrquery` package from a Google Colab notebook.

In [0]:
## source: https://github.com/r-lib/httr/pull/634
reassignInPackage("is_interactive", pkgName = "httr", function() {return(TRUE)})

The following command will setup access to BigQuery. Sign in with the Google account that you provided the organizers of the event (to guarantee access to the MIMIC dataset). Follow the link and copy and paste the authorization code in the prompt below.

In [160]:
bq_auth(use_oob = TRUE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=603366585132-0l3n5tr582q443rnomebdeeo0156b2bc.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code



Enter authorization code: 4/xAFynLJjmub0etyijLEfLzWj2EOvQP5Lf2yTNJ_gnq5aLHZFak1gXW4


We'll also set the project name that will be used to run queries and access MIMIC data for this datathon on Google BigQuery.

In [0]:
projectid = "bidmc-datathon"

## Query to BigQuery

Now we can start exploring the data. 

Our dataset is stored on BigQuery, Google's database engine. Since this notebook is running the R kernel, *unfortunately*, we won't be able to use some of the nice Colab notebook magic functions to run BigQuery SQL queries directly in a cell. **Fortunately**, querying data from BigQuery using R's `bigrquery` package is still relatively straightforward! Queries are written in SQL, a common language for extracting data from databases. The structure of an SQL query is:

```sql
SELECT <columns>
FROM <table>
WHERE <criteria, optional>
```

`*` is a wildcard that indicates all columns. For more details on syntax for writing SQL queries, check out the [BigQuery query syntax guide](https://cloud.google.com/bigquery/docs/reference/standard-sql/query-syntax).

To run an SQL query in R, simply turn the query into a string and call the `bq_project_query` function to *execute* the query and `bq_table_download` to *download* the results.

Here's a simple example showing how to download all rows (`*`) from an example dataset called `patients` in the `mimiciii_demo` project. 

*Remember to include your `projectid` defined above!*

In [0]:
demo_sql <- "
SELECT *
FROM `physionet-data.mimiciii_demo.patients`
"

## execute SQL 
demo_patients <- bq_project_query(projectid, query = demo_sql)

## download results
demo_patients <- bq_table_download(demo_patients)

We can now inspect our downloaded dataset.

In [164]:
head(demo_patients)

ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
<int>,<int>,<chr>,<dttm>,<dttm>,<dttm>,<dttm>,<int>
9467,10006,F,2094-03-05,2165-08-12,2165-08-12,2165-08-12,1
9472,10011,F,2090-06-05,2126-08-28,2126-08-28,NA,1
9474,10013,F,2038-09-03,2125-10-07,2125-10-07,2125-10-07,1
9478,10017,F,2075-09-21,2152-09-12,NA,2152-09-12,1
9486,10026,F,NA,2195-11-24,NA,2195-11-24,1
9487,10027,F,2108-01-15,2190-09-14,NA,2190-09-14,1


Great! That wasn't too bad at all!

We'll be mixing a fair bit of SQL in our cohort and feature extraction code below to gather data from various tables in the MIMIC-III database. Details about the different tables that we'll be working with are available on the [MIMIC site](https://mimic.physionet.org/gettingstarted/overview/) and for derived tables (some referenced below), the [`MIMIC-code` GitHub repo](https://github.com/MIT-LCP/mimic-code/tree/master/concepts).

We'll write a quick wrapper to download our data since there's no real need to call the "query" and "download" functions separately

In [0]:
bq_runsql <- function(query) {
  bq_table_download(bq_project_query(projectid, query))
}

We can now just call `bq_runsql()` in future code chunks to execute our queries!

# Cohort extraction


## Define cohort

We'll start by extracting our cohort and features from a few different tables in the MIMIC-III database.

We'll be restricting our analysis to patient records for hospital admissions that meet the following criteria:
- patient expired during the admission
- patient record is not an "organ donor" entry
- patient record is from [MetaVision](https://mimic.physionet.org/mimicdata/metavision/) system (vs. [CareVue](https://mimic.physionet.org/mimicdata/carevue/))
- patient language is recorded
- patient chart events are available in database

For each hospital admission, features of the patient and stay are collected from the following clinical MIMIC databases:
- `physionet-data.mimiciii_clinical.icustays`
- `physionet-data.mimiciii_clinical.admissions`
- `physionet-data.mimiciii_clinical.patients`

Again, check the [MIMIC site](https://mimic.physionet.org/gettingstarted/overview/) and [`MIMIC-code` GitHub repo](https://github.com/MIT-LCP/mimic-code/tree/master/concepts) for more information about each of the clinical and derived tables.

In [0]:
sql_cohort <- 
"
    SELECT DISTINCT 
        ic.subject_id,
        ic.hadm_id,
        ic.icustay_id,
        CASE WHEN ad.language='ENGL' THEN 1 ELSE 0 END AS english,
        ROUND(CAST(LEAST(
            DATETIME_DIFF(ic.intime, pt.dob, SECOND) / (60*60*24*365.242),
            91.4) AS NUMERIC), 1
        ) AS age,
        CASE
            WHEN pt.gender='M' THEN 1
            WHEN pt.gender='F' THEN 0
            ELSE NULL
        END AS male,
        ad.ethnicity,
        ad.marital_status,
        ad.religion,
        ad.insurance,
        ad.diagnosis,
        ad.admission_type,
        ad.admission_location,
        ad.admittime AS hosp_admittime,
        ad.dischtime AS hosp_dischtime,
        ROUND(CAST(DATETIME_DIFF(ad.dischtime, ad.admittime, SECOND) / (60*60*24)
            AS NUMERIC), 4
        ) AS los_hospital,
        ic.first_careunit,
        ic.intime AS icu_intime,
        ic.outtime AS icu_outtime,
        ic.los AS los_icu
    FROM `physionet-data.mimiciii_clinical.icustays` ic
        INNER JOIN `physionet-data.mimiciii_clinical.admissions` ad 
            ON ad.hadm_id = ic.hadm_id
        INNER JOIN `physionet-data.mimiciii_clinical.patients` pt 
            ON ic.subject_id = pt.subject_id
    WHERE ad.hospital_expire_flag = 1
        AND LOWER(ad.diagnosis) NOT LIKE 'organ donor account%'
        AND ic.dbsource = 'metavision'
        AND ad.language IS NOT NULL
        AND ad.has_chartevents_data = 1
    ORDER BY 1
"

bq_cohort <- bq_runsql(sql_cohort)

The above table includes multiple ICU stays (`icustay_id`) for each hospital admission (`hadm_id`). For the purposes of our analysis, we will only be looking at the *first* ICU stay of each hospital admission. 

While we could do directly in R, we'll determine the set of `icustay_id`s which correspond to the first ICU stay of each hospital admission using a query to BigQuery.

In [0]:
sql_firststays <- 
"
SELECT FIRST_VALUE(icustay_id) OVER w AS icustay_id
FROM `physionet-data.mimiciii_clinical.icustays`
WINDOW w AS (PARTITION BY hadm_id ORDER BY intime ASC)
"

bq_firststays <- bq_runsql(sql_firststays)

## make distinct since table includes duplicated rows
bq_firststays <- dplyr::distinct(bq_firststays)

Now, we use an `inner_join` to subset our cohort to just the first stay of each admission.

In [0]:
cohort <- bq_cohort %>%
    dplyr::inner_join(bq_firststays, by = "icustay_id")

Check the number of ICU stays we have in our cohort.

In [175]:
dim(cohort)

[1] 2285   20

That's still a sizeable number of observations!

# Extraction (prelude)

Now that we've extracted and defined our primary patient cohort along with several patient-level features, we will add additional features/covariates pulled from separate tables (vital & lab values, sepsis-3, ventilation status) to control for additioadmissions-levelcovariates and analyze our outcomes (e.g. invasive procedures) during modelling. 

But why controlling for covariates? 

Our problem definition is to analyze whether English vs. non-English speaking patients received different (or more invasive) treatment. What if a patient needed more invasive treatments (such as ventilation) because of other reasons, e.g. organ failure at admission? To account for this, we need to add covariates (variables capturing the state of a patient) to our model to adjust for whether a treatment was ordered due to a more severe patient state when studying the relationship with language.

The MIMIC repository fortunately already provides some SQL code and derived tables for extracting these variables/outcomes (e.g. vitals, labs, ventilation events). Before running those extraction SQL queries and building our design matrix, we'll quickly cover how features like blood pressure, heart rate etc. are actually extracted from MIMIC.

### `chartevents` and _ItemIDs_
MIMIC has two central tables which contain most of the structured vital and lab data called:

- `physionet-data.mimiciii_clinical.chartevents` (`chartevents`) and
- `physionet-data.mimiciii_clinical.labevents` (`labevents`).

Those tables have roughly the following schema:

| Patient ID | Admission ID | ICU Stay ID | Timestamp    | ItemID | Value | Unit               |
|------------|--------------|-------------|--------------|--------|-------|--------------------|
| 1          | 1            | 1           | Jan 1st, 1pm | 1      | 80    | bpm                |
| 2          | 2            | 2           | Jan 2nd, 2pm | 2      | 13    | breaths per minute |


The first three columns specify:
- the *patient*, 
- the *hospital admission* (a patient could have multiple hospital admissions) and 
- the *ICU admission* (there can be multiple ICU admissions during each hospital stay). 

The fourth column specifies the administration timestamp when the the event occurred or when the lab value was entered into the EHR. 

Columns 5 to 7 contain the most important part of each row: the actual value which is charted in this row. For example, _ItemID_: `1` in the first row in the table above denotes the heart rate for _patientID_: `1`, charted at `Jan 1st, 1pm`, which is `80` `bpm`. The next row denotes denotes a patient who had a respiration rate (_itemID_: `2`) of `13` `breaths per minute` at `Jan 2nd, 2pm`. 

You might be wondering how we knew _itemID_: `1` corresponded to "heart rate". Good question! This information is stored in a separate `d_items` table in MIMIC-III which maps from _itemID_ values to _labels_. We'll explore this more below.

Let's take a quick look at some of the real entries in the `chartevents` table in MIMIC-III. (We're using the `LIMIT` statement in our SQL query to only show the first 10 entries in this table.)

In [176]:
sql <- "
SELECT subject_id, hadm_id, icustay_id, charttime, itemid, value, valueuom
FROM `physionet-data.mimiciii_clinical.chartevents`
ORDER BY charttime
LIMIT 10
"

bq_runsql(sql)

subject_id,hadm_id,icustay_id,charttime,itemid,value,valueuom
<int>,<int>,<int>,<dttm>,<int>,<chr>,<chr>
82574,118464,202134,2100-06-07 20:00:00,226543,Roman Catholic,NA
82574,118464,202134,2100-06-07 20:00:00,226544,English,NA
82574,118464,202134,2100-06-07 20:00:00,226381,Single,NA
82574,118464,202134,2100-06-07 20:00:00,227378,CC6D,NA
82574,118464,202134,2100-06-07 20:26:00,226381,Single,NA
82574,118464,202134,2100-06-07 20:26:00,226543,Roman Catholic,NA
82574,118464,202134,2100-06-07 20:26:00,226544,English,NA
82574,118464,202134,2100-06-07 23:06:00,226381,Single,NA
82574,118464,202134,2100-06-07 23:06:00,226543,Roman Catholic,NA


In the first 10 rows that we downloaded, you can see that those entries/item ids probably encode the religion, the marital state and the language of a patient.

# Feature extraction

## First day vitals/labs

Before we start to extract vital/labs for each patient, we need to answer two different questions: 
1. which item ids correspond to which vital/lab value in _chartevents_ and _labevents_, and 
2. in which time window do we want to extract those items.

While the second question depends on the actual context of the problem definition, the first one can be answered by looking into two other important tables in MIMIC: `d_items` (mentioned above) and `d_labitems`. Those tables contain the name of each vital and lab value charted/administered in the Beth Isreal ICU and map those names to their corresponding item ids. Let's have a look how one of those tables is structured:

In [177]:
sql <- "
SELECT itemid, label
FROM `physionet-data.mimiciii_clinical.d_items`
ORDER BY itemid
LIMIT 10"

bq_runsql(sql)

itemid,label
<int>,<chr>
1,% Inspir. Time
2,ABI (L)
3,ABI (R)
4,ABI Ankle BP [Right]
5,ABI Brachial BP [Right]
6,ABP [Systolic]
24,ART Lumen Volume
25,AV Interval
26,AaDO2


In [178]:
sql <- "
SELECT itemid, label
FROM `physionet-data.mimiciii_clinical.d_labitems`
ORDER BY itemid
LIMIT 10"

bq_runsql(sql)

itemid,label
<int>,<chr>
50800,SPECIMEN TYPE
50801,Alveolar-arterial Gradient
50802,Base Excess
50803,"Calculated Bicarbonate, Whole Blood"
50804,Calculated Total CO2
50805,Carboxyhemoglobin
50806,"Chloride, Whole Blood"
50807,Comments
50808,Free Calcium


As you can see, this table contains even the smallest events which can occur in the `chartevents` / `labevents` tables. The columns in those table, obviously, now map each vital/drug name to one _itemid_ which can now be extracted from the `chartevents` (for the `d_items` table) and `labevents` (for the `d_labitems` table) tables. 

Let's say we now want to extract the Inspiration Time (%) (first entry in the `d_items` table above) from `chartevents`. 

We could use the following SQL code to extract those events:

In [179]:
sql <- "
SELECT subject_id, hadm_id, icustay_id, charttime, itemid, value, valueuom
FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE itemid = 1
ORDER BY charttime
LIMIT 10"

bq_runsql(sql)

subject_id,hadm_id,icustay_id,charttime,itemid,value,valueuom
<int>,<int>,<int>,<dttm>,<int>,<chr>,<chr>
20957,113808,246035,2100-06-26 17:56:00,1,33%,NA
20957,113808,246035,2100-06-26 22:00:00,1,NA,NA
20374,133497,269025,2100-08-16 13:17:00,1,25%,NA
20374,133497,269025,2100-08-16 15:00:00,1,25%,NA
11425,194366,288272,2100-09-09 14:00:00,1,33%,NA
11425,194366,288272,2100-09-09 19:00:00,1,33%,NA
11425,194366,288272,2100-09-09 22:16:00,1,33%,NA
11425,194366,288272,2100-09-10 04:00:00,1,33%,NA
11425,194366,288272,2100-09-10 06:00:00,1,25%,NA


As you can see, we now filtered the `chartevents` table for the first 10 inspiration rates (%) for multiple patients. Note that the charttimes have been shifted to the future (year 2100) to de-identify the patients in MIMIC.

Extracting reliable item IDs from MIMIC for, e.g. extracting heart rates, respiration rates etc. is a really tedious task since sometimes there are multiple item ids describing the same vital feature, so bear in mind that collecting item IDs for particular features can take a lot of time! 

Since we don't have enough time today to collect all of those co-variates manually, we fortunately can use ready-to-use MIMIC SQL code for that purpose. In the official MIMIC code repository [HERE](https://github.com/MIT-LCP/mimic-code) (under the _concepts_ folder) you can find tons of SQL code which already extracts vitals, labs, dialysis events, mechanical ventilation data etc. - and that's why we now want to make use of that code! 

Fortunately, all those SQL queries have already been executed by the MIT team and have been added to the `mimiciii_derived` table in the BigQuery MIMIC dataset, so we don't need to re-run those queries. For instance, the SQL query [`vitals-first-day.sql`](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/firstday/vitals-first-day.sql) already extracts all relevant vital values for the first day of each ICU admission in MIMIC, so let's use the resulting table for our own cohort. This is the output of the table when limiting the query to the first 10 rows:

In [180]:
sql <- "
SELECT *
FROM `physionet-data.mimiciii_derived.vitalsfirstday`
LIMIT 10"

bq_runsql(sql)

subject_id,hadm_id,icustay_id,HeartRate_Min,HeartRate_Max,HeartRate_Mean,SysBP_Min,SysBP_Max,SysBP_Mean,DiasBP_Min,⋯,RespRate_Mean,TempC_Min,TempC_Max,TempC_Mean,SpO2_Min,SpO2_Max,SpO2_Mean,Glucose_Min,Glucose_Max,Glucose_Mean
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,163353,243653,140,140,140.00000,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,145834,211552,75,168,111.78571,64,217,102.9600,28,⋯,17.36111,36.20000,37.60000,37.00288,74,100,97.87097,136,306,232.4167
4,185777,294638,74,111,89.21739,97,139,118.0000,55,⋯,25.35294,36.33333,37.44445,36.84445,91,100,97.50000,153,266,199.6667
6,107064,228232,76,100,84.16000,127,187,153.6471,40,⋯,12.32000,36.00000,37.33333,36.51515,84,100,99.20000,102,181,148.5000
7,118037,236754,115,121,118.00000,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,159514,262299,125,154,138.29630,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
9,150750,220597,82,111,92.50000,106,217,159.3750,53,⋯,14.36170,35.50000,37.88889,37.04938,95,100,97.68293,151,178,161.2857
10,184167,288409,128,162,147.56000,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
11,194540,229441,70,101,84.95833,91,118,105.1667,38,⋯,18.04545,36.33333,37.38889,36.80556,93,99,96.50000,120,149,129.8750


And the output for `labsfirstday`.

In [181]:
sql <- "SELECT *
FROM `physionet-data.mimiciii_derived.labsfirstday`
LIMIT 10"

bq_runsql(sql)

subject_id,hadm_id,icustay_id,ANIONGAP_min,ANIONGAP_max,ALBUMIN_min,ALBUMIN_max,BANDS_min,BANDS_max,BICARBONATE_min,⋯,INR_min,INR_max,PT_min,PT_max,SODIUM_min,SODIUM_max,BUN_min,BUN_max,WBC_min,WBC_max
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,163353,243653,NA,NA,NA,NA,1,1,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,0.1,22.0
3,145834,211552,15,23,1.8,1.8,2,5,11,⋯,1.3,1.7,13.5,15.7,136,153,41,53,11.3,24.4
4,185777,294638,15,15,2.8,2.8,NA,NA,21,⋯,1.1,1.1,12.8,12.8,141,141,10,10,9.7,9.7
5,178980,214757,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,13.9,13.9
6,107064,228232,20,23,3.0,3.0,NA,NA,15,⋯,1.0,1.4,12.6,14.6,134,138,62,65,10.6,10.6
7,118037,236754,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,22.8,22.8
7,118037,278444,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
8,159514,262299,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,18.7,18.7
9,150750,220597,13,13,NA,NA,NA,NA,26,⋯,1.1,1.1,12.7,12.7,136,140,16,17,7.5,13.7


*Note that it's sufficent for our problem definition to only use the first admission day vitals and labs since using all values for each ICU stays would basically translate into a time-series problem which is a completely different game than just using one data row per patient (the mean, max and min values of each vital/lab already contain much valuable information!)*

*Also, you might have notices that many entries are NaN, so missing values. You need to come up with a strategy during inference, how you want to treat/impute those values. Missingness is one of the biggest challenges in working with EHR data.*

After now finding out how to extract vitals and labs using the ready-to-use SQL code from the MIMIC repository, we now need to create a larger table which concats all of those lab/vital features for each ICU stay of each patient (we later limit our cohort to only the first ICU stay of each patient).
The following SQL code queries merges (_JOIN_ s) the different lab/vital tables and outputs one big table which contains first day min, max and mean lab and vitals for _each_ patient in MIMIC (we still need to limit it to our cohort). We store the resulting table in the variable `bq_labvital`.

In [0]:
sql_labvital <- "
WITH t1 AS (
SELECT labs.*, 
    HeartRate_Min, HeartRate_Max, HeartRate_Mean, 
    SysBP_Min, SysBP_Max,	SysBP_Mean,	
    DiasBP_Min,	DiasBP_Max,	DiasBP_Mean,
    MeanBP_Min,	MeanBP_Max,	MeanBP_Mean,
    RespRate_Min,	RespRate_Max,	RespRate_Mean,	
    TempC_Min, TempC_Max,	TempC_Mean,	
    SpO2_Min,	SpO2_Max,	SpO2_Mean
FROM `physionet-data.mimiciii_derived.labsfirstday` labs
LEFT JOIN `physionet-data.mimiciii_derived.vitalsfirstday` vitals
  ON vitals.icustay_id = labs.icustay_id
)
, firstadmissions_1 AS (
  SELECT *, ROW_NUMBER() OVER (PARTITION BY subject_id ORDER BY intime) AS rowcnt
  FROM `physionet-data.mimiciii_clinical.icustays`
), firstadmissions_2 AS (
  SELECT *
  FROM firstadmissions_1
  WHERE rowcnt = 1
)
SELECT *
FROM t1
WHERE t1.icustay_id 
IN (
  SELECT DISTINCT icustay_id 
  FROM firstadmissions_2 
  WHERE NOT icustay_id is NULL
  )"

bq_labvital <- bq_runsql(sql_labvital)

In [183]:
head(bq_labvital)

subject_id,hadm_id,icustay_id,ANIONGAP_min,ANIONGAP_max,ALBUMIN_min,ALBUMIN_max,BANDS_min,BANDS_max,BICARBONATE_min,⋯,MeanBP_Mean,RespRate_Min,RespRate_Max,RespRate_Mean,TempC_Min,TempC_Max,TempC_Mean,SpO2_Min,SpO2_Max,SpO2_Mean
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,145834,211552,15,23,1.8,1.8,2,5,11,⋯,75.69281,5,24,17.36111,36.20000,37.60000,37.00288,74,100,97.87097
6,107064,228232,20,23,3.0,3.0,NA,NA,15,⋯,84.53922,8,20,12.32000,36.00000,37.33333,36.51515,84,100,99.20000
21,109451,217847,23,27,NA,NA,NA,NA,15,⋯,74.11538,12,26,16.79167,35.38889,37.44445,36.18182,95,100,99.79167
38,185910,248910,16,17,2.7,2.7,NA,NA,20,⋯,73.00001,14,31,24.32500,34.61111,37.88889,36.52222,68,100,96.83784
59,104130,224440,21,24,NA,NA,NA,NA,19,⋯,72.15151,9,24,16.40909,36.38889,37.33333,36.87037,94,100,97.10526
68,170467,294232,16,16,NA,NA,NA,NA,22,⋯,60.52777,20,35,27.58333,36.00000,37.55555,36.91667,90,100,95.62500


Now we've extracted a design matrix which contains all first day lab and vital statistics for the _complete_ MIMIC population. We'll later limit those patients to our cohort, but before that we keep adding more features to our design matrix (this can be both covariates and outcomes)

Note that this table of first day labs and vitals spans many more ICU stays and hospital admissions than our cohort extracted and defined above.


In [184]:
dim(bq_labvital)

[1] 46476    62

We are only interested in the labs and vitals for our cohort. We add this to our existing `cohort` table by `left_join`ing on `subject_id`, `hadm_id`, and `icustay_id`.


In [0]:
cohort <- cohort %>%
    dplyr::left_join(bq_labvital, by = c("subject_id", "hadm_id", "icustay_id"))

## Severity scores (SAPS-II)

Severity score are great features since they translate clinical knowledge into an understandable score which correlates with a patient's current physical state (at admission). These scores could for instance capture organ dysfunction, the inflammatory state of a patient or the consciousness of a patient. 

We will add the Simplified Acute Physiology Score II (SAPS II)to our table. You can later decide if you want to use this score to control for a patient's severity of illness.
SAPS-II has already been computed using the SQL code [HERE](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/severityscores/sapsii.sql). We will simply download the derived table already available on BigQuery and add it to our feature table (variable: `sapsii`). 

Please note that we'll merge the resulting `bq_sapsii` table later to our vital/labs design matrix we derived earlier.

In [0]:
bq_tab_sapsii <- "physionet-data.mimiciii_derived.sapsii"

## no need to execute an SQL query - just download!
bq_sapsii <- bq_table_download(bq_tab_sapsii)

Again, this table covers many more admissions and stays.

In [187]:
dim(bq_sapsii)

[1] 61532    20

We again join by subject, admissions and ICU stay IDs.

In [0]:
cohort <- cohort %>%
    dplyr::left_join(bq_sapsii, by = c("subject_id", "hadm_id", "icustay_id"))


## Comorbidity scores (Elixhauser)

When doing the inference analysis, it is also important to control for comorbidities of the patients. Since there are many comorbidity scores (e.g. Elixhauser), MIMIC already contains code that not only extracts comorbidities (binary flags, e.g. renal failure) by using the ICD-9 diagnosis codes, but also computes different comorbidity scores (e.g. Elixhauser QUAN). 

We will extract various binary comorbidity flags and a few Elixhauser scores from the follow tables to add to our cohort table.
- `physionet-data.mimiciii_derived.elixhauser_quan`
- `physionet-data.mimiciii_derived.elixhauser_quan_score`


In [0]:
sql_comorb <- "
    SELECT DISTINCT 
        el.hadm_id,
        el.congestive_heart_failure,
        el.cardiac_arrhythmias,
        el.valvular_disease,
        el.pulmonary_circulation,
        el.peripheral_vascular,
        el.hypertension,
        el.paralysis,
        el.other_neurological,
        el.chronic_pulmonary,
        el.diabetes_uncomplicated,
        el.diabetes_complicated,
        el.hypothyroidism,
        el.renal_failure,
        el.liver_disease,
        el.peptic_ulcer,
        el.aids,
        el.lymphoma,
        el.metastatic_cancer,
        el.solid_tumor,
        el.rheumatoid_arthritis,
        el.coagulopathy,
        el.obesity,
        el.weight_loss,
        el.fluid_electrolyte,
        el.blood_loss_anemia,
        el.deficiency_anemias,
        el.alcohol_abuse,
        el.drug_abuse,
        el.psychoses,
        el.depression,
        elscore.elixhauser_vanwalraven,
        elscore.elixhauser_sid29,
        elscore.elixhauser_sid30
    FROM `physionet-data.mimiciii_derived.elixhauser_quan` el 
        INNER JOIN `physionet-data.mimiciii_derived.elixhauser_quan_score` elscore 
            ON el.hadm_id = elscore.hadm_id
"

bq_comorb <- bq_runsql(sql_comorb)

We can take a quick look at the various comorbidities that we have available in this table.

In [190]:
head(bq_comorb)

hadm_id,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,other_neurological,chronic_pulmonary,⋯,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,elixhauser_vanwalraven,elixhauser_sid29,elixhauser_sid30
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
100001,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,5,5,6
100003,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,11,4,5
100006,0,1,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,1,11,11,22
100007,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,-1,-2
100009,0,1,0,0,1,1,0,0,0,⋯,0,0,0,0,0,0,0,-2,-2,5
100010,0,0,0,0,0,0,0,0,0,⋯,1,0,0,0,0,0,0,17,22,28


We now add these comorbidity flags and scores to our cohort table.

In [0]:
cohort <- cohort %>%
    dplyr::left_join(bq_comorb, by = "hadm_id")

## Sepsis status (sepsis-III)
Next, we want to add sepsis-III to our data table. Sepsis is a one of the major death causes in ICUs around the world, so it's really important to control for sepsis-markers during modelling. Sepsis-III is the most recent definition to define if a patient is having sepsis and adopted in many data-driven projects. A occurence of sepsis is defined here as a SOFA score (organ dysfunction severity score) increase by 2 points and the occurence of a suspected infection (microbiologyevent and antibiotics subscription). Since it would take too long to extract those patients manually from MIMIC, we stick to some code from another [MIT-MIMIC repository](https://github.com/alistairewj/sepsis3-mimic).

In [0]:
sql_sepsis3 <- "
WITH t1 as
(
select ie.icustay_id, ie.intime

    -- suspicion of infection using POE
    , case when spoe.suspected_infection_time is not null then 1 else 0 end
        as suspected_of_infection_poe
    , spoe.suspected_infection_time as suspected_infection_time_poe
    , (UNIX_SECONDS(CAST(ie.intime as TIMESTAMP)) - UNIX_SECONDS(CAST(spoe.suspected_infection_time as TIMESTAMP)))
          / 60.0 / 60.0 / 24.0 as suspected_infection_time_poe_days
    , so.sofa as sofa
from `physionet-data.mimiciii_clinical.icustays` ie
inner join `physionet-data.mimiciii_clinical.admissions` adm
    on ie.hadm_id = adm.hadm_id
inner join `physionet-data.mimiciii_clinical.patients` pat
    on ie.subject_id = pat.subject_id
left join `physionet-data.mimiciii_derived.sofa` so
  on ie.icustay_id = so.icustay_id
left join `physionet-data.mimiciii_derived.suspinfect_poe` spoe
  on ie.icustay_id = spoe.icustay_id
)

SELECT DISTINCT icustay_id, 1 as sepsis_3, sofa
FROM t1 
WHERE  ((t1.suspected_infection_time_poe is not null
    and UNIX_SECONDS(TIMESTAMP(t1.suspected_infection_time_poe)) >= (UNIX_SECONDS(TIMESTAMP(t1.intime)) - 24*60*60)
  )
or (
        t1.suspected_infection_time_poe is not null
    and UNIX_SECONDS(TIMESTAMP(t1.suspected_infection_time_poe)) <= (UNIX_SECONDS(TIMESTAMP(t1.intime)) + 24*60*60)
  ))
  AND NOT t1.suspected_infection_time_poe_days IS NULL
  AND sofa >= 2
"

bq_sepsis3 <- bq_runsql(sql_sepsis3)

This table includes the subset of ICU stays that have been identified as being positive for sepsis according to the Sepsis-3 criteria described above, as well as the increase in SOFA score for this subset of ICU stays.

In [193]:
head(bq_sepsis3)

icustay_id,sepsis_3,sofa
<int>,<int>,<int>
214027,1,6
217934,1,7
224026,1,3
217847,1,7
290505,1,3
211885,1,7


We will now add these scores to our cohort table and set the `sepsis_3` value to "0" for ICU stays not in the Sepsis-3 table created above (these ICU stays were not identified as sepsis positive).

In [0]:
cohort <- cohort %>%
    dplyr::left_join(bq_sepsis3, by = "icustay_id") %>%
    tidyr::replace_na(list(sepsis_3 = 0L))

## Ventilation status

Ventilator status is extracted from the `ventdurations` table. This derived table was created using the code available on the `MIMIC-code` GitHub repo [here](https://github.com/MIT-LCP/mimic-code/blob/master/concepts/durations/ventilation-durations.sql). 


In [0]:
bq_tab_ventdur <- "physionet-data.mimiciii_derived.ventdurations"

## no need to execute an SQL query - just download!
bq_ventdur <- bq_table_download(bq_tab_ventdur)

The `ventdurations` table includes durations for all predicted patient ventilation events included in the MIMIC database (see the above link for how this is determined). This means the table includes multiple ventilation durations (events) for each patient and each stay. 

We will collapse the table to individual ICU stays (`icustay_id`). 

In [0]:
ventdur_bystay <- bq_ventdur %>%
    dplyr::filter(!is.na(icustay_id)) %>%
    dplyr::group_by(icustay_id) %>%
    dplyr::summarize(mechvent_days = sum(duration_hours) / 24,
                     mechvent_start = min(starttime)) %>%
    dplyr::mutate(mechvent_event = 1L)

For each ICU stay, we now have the number of days on mechanical ventilation and the start of ventilation. (We also now know which ICU stays had ventilation events - the ones in the table.)

We now add these ventilation features to our cohort.

In [0]:
cohort <- cohort %>%
    dplyr::left_join(ventdur_bystay, by = "icustay_id")

Since our ventilation table only include ICU stays *with* ventilation events, the other ICU stays in the table will have `NAs` in the ventilation columns. We'll replace these with "0s".

In [0]:
cohort <- cohort %>%
    dplyr::mutate(mechvent_event = ifelse(is.na(mechvent_event), 0, mechvent_event),
                  mechvent_days = ifelse(is.na(mechvent_days), 0, mechvent_days))

While we have ventilation status and duration, what we're actually interested in is the ventilation *free days*. We compute the ventilation free days as the number of days without ventilation *after* ventilation is started. This is calculated using the ICU "out time", ventilation start time, and ventilation day columns.

In [0]:
cohort <- cohort %>%
    dplyr::mutate(
        mechvent_candidate_days = as.numeric(icu_outtime - mechvent_start, unit = "days"),
        mechvent_free_days      = mechvent_candidate_days - mechvent_days)

We will drop the columns that we're no longer interested in using.

In [0]:
cohort <- cohort %>%
    dplyr::select(-mechvent_days, -mechvent_candidate_days, -mechvent_start)

# Save cohort

Finally, we'll write our complete cohort table to Google Drive. To do this, we will use the [`googledrive` package](https://googledrive.tidyverse.org/). There are several functions in this package (most prefixed with `drive_*`) that can be used to move, create, copy and remove files on Google Drive.

Let's get started!

## Connect to Drive

Connecting the Google Colab with Google Drive is a similar process to the one we used to connect to BigQuery above. First, we need to authenticate our account with the following command.

(Notice that here we call `drive_auth` and above we called `bq_auth`.)

In [201]:
drive_auth(use_oob = TRUE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=603366585132-0l3n5tr582q443rnomebdeeo0156b2bc.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code



Enter authorization code: 4/xAH_Fhm-LhqRHXm6HsdQ6gbPUBwB2zEvALyefMtn1uqXTOpE9HSMyog


Now that we have acces to Google Drive, we can check our connection.

To see all of the files at the home path of our Google Drive account (usually called "My Drive"), we can call the `drive_ls()` function. Remember to specify `path = "~"` when calling this function, otherwise the function will try to list out *all* of your files on Google Drive. (If you're like me, that can be a lot!!)

In [0]:
gd_files <- drive_ls(path = "~")

Let's take a look at what was returned.

In [209]:
show(gd_files)

# A tibble: 3 x 3
  name            id                                drive_resource   
* <chr>           <chr>                             <list>           
1 professional    0BwaR2ExkoRdua2xXV3l2QnZhanM      <named list [33]>
2 Colab Notebooks 1SpiGu6n32pft44rIK_lTe0scRqm58_V8 <named list [33]>
3 personal        0BwaR2ExkoRduVGx2alNvTmROY0k      <named list [33]>


This is a table with the `name` of the files, as well as the Google Drive `id`, and a complex `drive_resource` column. We can ignore these additional files.

If you visit your [Google Drive page](https://drive.google.com/), hopefully you'll see the same set as files as above.


## Save to Drive

The last step is to write our CSV file to Google Drive.

We'll do this in 2 steps:
- save a temporary CSV file here (in the Colab world),
- copy that file to our Google Drive account.

First, we write the table to a CSV file.

In [0]:
write_csv(cohort, "dataset_datathon_28022020.csv")

We can check to see that a file with the above name now exists.

In [211]:
list.files()

[1] "dataset_datathon_28022020.csv" "sample_data"

Next, we upload the file to Google Drive. The following command will upload our CSV file to our home folder on Google Drive ("My Drive").

In [212]:
drive_upload("dataset_datathon_28022020.csv")

Local file:
  * dataset_datathon_28022020.csv
uploaded into Drive file:
  * dataset_datathon_28022020.csv: 1WjNArdHJ8N6L1RCCv-rk6eY8btsdXuYy
with MIME type:
  * text/csv



Let's verify that the file is now uploaded to Google Drive by calling the same `drive_ls()` function as above.

In [213]:
show(drive_ls("~"))

# A tibble: 4 x 3
  name                         id                               drive_resource  
* <chr>                        <chr>                            <list>          
1 dataset_datathon_28022020.c… 1WjNArdHJ8N6L1RCCv-rk6eY8btsdXu… <named list [38…
2 professional                 0BwaR2ExkoRdua2xXV3l2QnZhanM     <named list [33…
3 Colab Notebooks              1SpiGu6n32pft44rIK_lTe0scRqm58_… <named list [33…
4 personal                     0BwaR2ExkoRduVGx2alNvTmROY0k     <named list [33…


Now that we have our cohort, we're ready to move on to our analysis.